### Membaca data

In [1]:
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)

import time

In [2]:
# Importing data
data = pd.read_csv('clean_data.csv')   
del data['Unnamed: 0']
# del data['0']
data

,1,10,100,101,102,103,104,105,106,107,...,91,92,93,94,95,96,97,98,99,radiant_win
0,0,-1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621059,0,0,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
621060,0,0,0,0,0,0,0,0,1,0,...,0,0,-1,0,0,0,0,0,0,0
621061,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
621062,0,0,-1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split dataset menjadi train, validation, dan test
from sklearn.model_selection import train_test_split
# train_inputs, test_inputs, train_targets, test_targets = train_test_split(X, y, test_size = 0.25, random_state = 0)
train_inputs, X_test, train_targets, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
val_inputs, test_inputs, val_targets, test_targets = train_test_split(X_test, y_test, test_size = 0.40, random_state = 0)

### Model

In [4]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import MaxNorm

#### Mendefinisikan model

In [5]:
# Set the output and hidden layer sizes
input_neuron= 121
hidden_neuron = 300
output_neuron = 1


init_mode = 'random_uniform'
dropout_rate = 0.4
weight_constraint = 3
    
# define how the model will look like
model = Sequential()
model.add(InputLayer(input_shape=(input_neuron,)))                                      # input layer
model.add(Dense(hidden_neuron, kernel_initializer=init_mode, activation='gelu',         # 1st hidden layer
                kernel_constraint=MaxNorm(weight_constraint)))
model.add(Dense(output_neuron, kernel_initializer=init_mode, activation='sigmoid'))     #  output layer

learning_rate = 0.001
momentum = 0.9

# Choose the optimizer and the loss function
optimizer = SGD(learning_rate=learning_rate, momentum=momentum)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

batch_size = 80
max_epochs = 10

# set an early stopping mechanism
# early_stopping = tf.keras.callbacks.EarlyStopping(
#     patience=10,
#     min_delta=0.01,
#     restore_best_weights=True,
# )

# red_lr= tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',patience=2,verbose=1,factor=0.1)

#### Training model

In [6]:
start_time = time.time()

# fit the model
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, 
#           callbacks=[red_lr,early_stopping], # early stopping
          validation_data=(val_inputs, val_targets), # validation data
          verbose = 2 
          )  

ex_time = time.time() - start_time
print("\n--- %s minutes %d seconds ---" % (int(ex_time/60), int(ex_time%60)))

Epoch 1/10
5823/5823 - 9s - loss: 0.6737 - accuracy: 0.5784 - val_loss: 0.6615 - val_accuracy: 0.5988
Epoch 2/10
5823/5823 - 8s - loss: 0.6522 - accuracy: 0.6162 - val_loss: 0.6451 - val_accuracy: 0.6262
Epoch 3/10
5823/5823 - 8s - loss: 0.6437 - accuracy: 0.6280 - val_loss: 0.6422 - val_accuracy: 0.6293
Epoch 4/10
5823/5823 - 8s - loss: 0.6421 - accuracy: 0.6295 - val_loss: 0.6415 - val_accuracy: 0.6298
Epoch 5/10
5823/5823 - 8s - loss: 0.6416 - accuracy: 0.6301 - val_loss: 0.6412 - val_accuracy: 0.6299
Epoch 6/10
5823/5823 - 8s - loss: 0.6414 - accuracy: 0.6304 - val_loss: 0.6411 - val_accuracy: 0.6298
Epoch 7/10
5823/5823 - 8s - loss: 0.6414 - accuracy: 0.6306 - val_loss: 0.6410 - val_accuracy: 0.6305
Epoch 8/10
5823/5823 - 8s - loss: 0.6413 - accuracy: 0.6306 - val_loss: 0.6410 - val_accuracy: 0.6304
Epoch 9/10
5823/5823 - 8s - loss: 0.6413 - accuracy: 0.6308 - val_loss: 0.6409 - val_accuracy: 0.6304
Epoch 10/10
5823/5823 - 8s - loss: 0.6413 - accuracy: 0.6306 - val_loss: 0.6409 - 

In [7]:
# model.summary()

#### Tes model

In [8]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

1941/1941 [==============================] - 2s 845us/step - loss: 0.6397 - accuracy: 0.6329

Test loss: 0.64. Test accuracy: 63.29%


### Menyimpan the model

In [9]:
# model.save("model.h5")